In [ ]:
# Row for install library
!pip install numpy confluent-kafka confluent_kafka

In [ ]:
# Row for import library
import os
import time
import pyspark
import findspark
from ast import literal_eval
from pyspark.sql import SparkSession
from confluent.schemaregistry.serializers import MessageSerializer
from confluent.schemaregistry.client import CachedSchemaRegistryClient

In [ ]:
# Row for init
findspark.init()
processName = 'MataElangLab'
brokers = os.environ['BOOTSTRAP_SERVERS']
kafkaTopic = 'netflowmeter'

In [ ]:
# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName(processName)
         # Add kafka package
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
         .getOrCreate())
sc = spark.sparkContext

In [ ]:
# Spark-Kafka Structured Stream
df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", brokers) # kafka server
  .option("subscribe", kafkaTopic) # topic
  .option("startingOffsets", "earliest") # start from beginning 
  .load())

In [ ]:
# Deserializer (If U using avro schema)
# schema_registry_client = CachedSchemaRegistryClient(url='http://xx.xxx.xxx:8081')
# serializer = MessageSerializer(schema_registry_client)

# simple decode to replace Kafka-streaming's built-in decode decoding UTF8 ()
# def decoder(s):
#     decoded_message = serializer.decode_message(s)
#     return decoded_message

dserialize = df.selectExpr("CAST(value AS STRING)")

In [ ]:
# Run RawData Query from readStream Using Foreach
# Print every row using a function

def print_row(row):
    print(row)

rawQuery = df.writeStream.foreach(print_row)

In [ ]:
# Get Value of Kafka readStream
valueQuery = ds.writeStream.foreach(print_row)